# Интуиция CLIP

В этом блоке мы **симулируем CLIP**. В качестве датасета возьмем объекты, у которых есть по две "модальности", каждая из которых представлена двумерным эмбеддингом. Эквивалентами таких эмбеддингов могут быть выходы маломерных энкодеров различных модальностей А и B, например: A — «картинка», B — «текст».


In [ ]:
import random
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print("Device:", device)

from dataclasses import dataclass

@dataclass
class Cfg:
    n_pairs: int = 256     # число пар (A,B)
    d: int = 2             # размерность эмбеддингов (2D для красивой визуализации)
    tau: float = 0.07      # температура (как в CLIP: logits / tau)
    epochs: int = 100
    vis_every: int = 20
    lr: float = 2e-2
    wd: float = 1e-4

cfg = Cfg()
cfg


### Обучаемые эмбеддинги вместо энкодеров

Вместо настоящих энкодеров заведём **два набора параметров** `A` и `B` (по вектору на каждую пару).  
Будем обучать их так, чтобы эмбеддинги внутри пары сближались, а вне — отталкивались.


In [ ]:
# обучаемые параметры — "выходы" визуального и текстового энкодеров
A = torch.nn.Parameter(torch.randn(cfg.n_pairs, cfg.d, device=device))
B = torch.nn.Parameter(torch.randn(cfg.n_pairs, cfg.d, device=device))

opt = torch.optim.AdamW([A, B], lr=cfg.lr, weight_decay=cfg.wd)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

print("A:", tuple(A.shape), "B:", tuple(B.shape))


### Контрастивные лоссы

**CLIP-стиль (InfoNCE / softmax):**
- Нормализуем векторы.
- Строим логиты как косинусные похожести / `tau`.
- Кросс-энтропия для обоих направлений (A→B и B→A) и усредняем.

**Sigmoid-стиль (в духе SigLIP):**
- Рассматриваем **все пары** независимо.
- Подаём все косинусные похожести в BCE-with-logits, где таргет — единицы на диагонали (позитивы) и нули вне диагонали (негативы).

In [ ]:
def clip_bidir_loss(a, b, tau: float):
    a = F.normalize(a, dim=-1)
    b = F.normalize(b, dim=-1)
    logits_ab = (a @ b.T) / tau
    logits_ba = (b @ a.T) / tau
    target = torch.arange(a.size(0), device=a.device)
    loss_ab = F.cross_entropy(logits_ab, target)
    loss_ba = F.cross_entropy(logits_ba, target)
    return 0.5 * (loss_ab + loss_ba), logits_ab

def sigmoid_pairwise_loss(a, b, scale: float = 10.0):
    a = F.normalize(a, dim=-1)
    b = F.normalize(b, dim=-1)
    sims = (a @ b.T) * scale
    targets = torch.eye(a.size(0), device=a.device)
    loss = F.binary_cross_entropy_with_logits(sims, targets)
    return loss, sims

Реализуем функцию, которая будет строить необходимые визуализации в процессе обучения.

Визуализирует эмбеддинги A и B с соединениями и статистикой.

    - Нормализуем для отображения

    - Отдельные scatter для A и для B (разные маркеры)

    - Совмещённый график с линиями между соответствующими парами
    
    - Распределение косинусных расстояний (1 - cos) для позитивных пар


In [ ]:
def visualize(a, b, epoch, loss_value, logits_ab=None, max_show=20, show_all=False):

    a_n = F.normalize(a, dim=-1).detach().cpu().numpy()
    b_n = F.normalize(b, dim=-1).detach().cpu().numpy()

    n_total = a_n.shape[0]
    n_show = n_total if show_all else min(max_show, n_total)
    colors = plt.cm.tab20(np.linspace(0, 1, n_show))

    # Вывод Top-1 A→B при наличии logits_ab
    if logits_ab is not None:
        with torch.no_grad():
            preds = logits_ab.argmax(dim=1)
            target = torch.arange(logits_ab.size(0), device=logits_ab.device)
            acc = (preds == target).float().mean().item()

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    ax1, ax2, ax3, ax4 = axes.flat

    for i in range(n_show):
        ax1.scatter(a_n[i, 0], a_n[i, 1],
                    c=[colors[i]], s=100, marker="o",
                    edgecolors="black", linewidth=1, alpha=0.8)
        if n_show <= 20:
            ax1.text(a_n[i, 0], a_n[i, 1] + 0.05, f"A{i+1}",
                     ha="center", fontsize=8, fontweight="bold")
    ax1.add_patch(plt.Circle((0, 0), 1, fill=False, ls="--", alpha=0.5, color="gray"))
    ax1.set_title(f"A")
    ax1.set_xlim(-1.1, 1.1); ax1.set_ylim(-1.1, 1.1)
    ax1.set_aspect("equal"); ax1.grid(alpha=0.3)

    for i in range(n_show):
        ax2.scatter(b_n[i, 0], b_n[i, 1],
                    c=[colors[i]], s=100, marker="s",
                    edgecolors="black", linewidth=1, alpha=0.8)
        if n_show <= 20:
            ax2.text(b_n[i, 0], b_n[i, 1] + 0.05, f"B{i+1}",
                     ha="center", fontsize=8, fontweight="bold")
    ax2.add_patch(plt.Circle((0, 0), 1, fill=False, ls="--", alpha=0.5, color="gray"))
    ax2.set_title(f"B")
    ax2.set_xlim(-1.1, 1.1); ax2.set_ylim(-1.1, 1.1)
    ax2.set_aspect("equal"); ax2.grid(alpha=0.3)

    for i in range(n_show):
        ax3.scatter(a_n[i, 0], a_n[i, 1],
                    c=[colors[i]], s=80, marker="o",
                    edgecolors="black", linewidth=1, alpha=0.85, label="A" if i==0 else "")
        ax3.scatter(b_n[i, 0], b_n[i, 1],
                    c=[colors[i]], s=80, marker="s",
                    edgecolors="black", linewidth=1, alpha=0.85, label="B" if i==0 else "")
        ax3.plot([a_n[i, 0], b_n[i, 0]],
                 [a_n[i, 1], b_n[i, 1]],
                 "k--", alpha=0.35, linewidth=0.7)
    ax3.add_patch(plt.Circle((0, 0), 1, fill=False, ls="--", alpha=0.5, color="gray"))
    ax3.set_title(f"Совмещённые пары\nTop-1 A→B accuracy: {acc*100:.2f}%")
    ax3.set_xlim(-1.1, 1.1); ax3.set_ylim(-1.1, 1.1)
    ax3.set_aspect("equal"); ax3.grid(alpha=0.3)
    ax3.legend(loc="lower left")

    # --- (1,1) распределение косинусных расстояний 1 - cos(A_i, B_i) ---
    sims = (F.normalize(a, dim=-1) @ F.normalize(b, dim=-1).T).detach().cpu().numpy()
    diag_sims = np.diag(sims)
    cos_dists = 1.0 - diag_sims
    ax4.hist(cos_dists, bins=20, alpha=0.8, color="skyblue", edgecolor="black")
    mean_dist = float(np.mean(cos_dists))
    std_dist = float(np.std(cos_dists))
    ax4.axvline(mean_dist, color="red", ls="--", lw=2, label=f"среднее={mean_dist:.3f}")
    ax4.set_title("Распределение косинусных расстояний")
    ax4.set_xlabel("1 - cos(A_i, B_i)")
    ax4.set_ylabel("Частота")
    ax4.grid(alpha=0.3)
    ax4.legend()

    fig.suptitle(f"Эпоха {epoch} | loss={loss_value:.4f}", fontsize=14)
    plt.tight_layout()
    plt.show()



Обучаем `A` и `B` так, чтобы A↔B совпадали. Каждые несколько эпох смотрим на геометрию и метрики.


In [ ]:
loss_history = []

for epoch in range(1, cfg.epochs + 1):
    opt.zero_grad()

    loss, logits_ab = clip_bidir_loss(A, B, cfg.tau)
    loss.backward()
    #torch.nn.utils.clip_grad_norm_([A, B], max_norm=1.0)
    opt.step()
    sched.step()

    loss_history.append(loss.item())

    if epoch % cfg.vis_every == 0 or epoch in (1, cfg.epochs):
        visualize(A, B, epoch, loss.item(), logits_ab=logits_ab)

# В конце обучения рисуем график потерь
plt.figure(figsize=(10,4))
plt.plot(loss_history)
plt.yscale("log")
plt.title("История обучения")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.grid(alpha=0.3)
plt.show()


# Zero-shot

Обычно для классификации нужно обучать модель на размеченном датасете. Но CLIP позволяет делать **zero-shot**:  
- Мы задаём список текстовых описаний (например, "футболка", "ноутбук", "чашка").  
- Модель превращает текст и изображения в одно пространство эмбеддингов.  
- Класс для картинки выбирается как тот текст, чей эмбеддинг ближе всего к эмбеддингу изображения.  

Этот подход позволяет модели без дополнительного обучения решать задачи классификации и поиска.  


In [ ]:
# Установка и загрузка зависимостей (если в colab/kaggle)
!pip install datasets transformers accelerate faiss-cpu umap-learn plotly seaborn -q

In [ ]:
import io
import random
import re
import collections
import pathlib
from math import ceil

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from datasets import load_dataset
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

import requests
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from tqdm.auto import tqdm

import umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import plotly.express as px

- **Датасет**: используем подмножество Amazon Products (5k примеров), так как это реальные изображения товаров с категориями. Такой датасет позволяет тестировать zero-shot классификацию и retrieval.  
- **Модель CLIP**: берём `openai/clip-vit-base-patch32`.  
  - ViT-B/32 — базовая версия, работает быстрее, чем более тяжёлые варианты.  
  - Для серьёзных задач можно заменить на SigLIP или большие версии CLIP.  
- **CLIPProcessor**: включает в себя токенизацию текста и препроцессинг изображений (resize, crop, normalization).  
- **Устройство**: проверяем доступность CUDA (NVIDIA GPU) или MPS (Apple Silicon).  


In [ ]:
device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.backends.mps.is_available() else "cpu"
)
print("device:", device)

dataset = load_dataset("milistu/AMAZON-Products-2023", split="train[:5000]")
print("dataset:", dataset)

MODEL_NAME = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(MODEL_NAME).to(device)
clip_processor = CLIPProcessor.from_pretrained(MODEL_NAME)
print("CLIP загружен:", MODEL_NAME)

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# Размер подмножества для демо (адаптируется к среде)
BATCH_SIZE = 8
N_SAMPLES = int(min(2000, len(dataset)))
BATCH_EMB = max(4, BATCH_SIZE * (2 if device == 'cpu' else 4))

print(f"SEED={SEED} | N_SAMPLES={N_SAMPLES} | BATCH_EMB={BATCH_EMB} | device={device}")

`to_pil`: универсальная функция для приведения изображения к формату `PIL.Image`. Поддерживает строки (пути, URL), numpy-массивы и байты.  
`display_images`: удобная функция для вывода сразу нескольких изображений в ряд. Это поможет наглядно показывать примеры при zero-shot классификации и retrieval.  


In [ ]:
def to_pil(img):
    "Универсально приводим к PIL.Image RGB."
    if isinstance(img, Image.Image):
        return img.convert("RGB")
    if isinstance(img, bytes):
        return Image.open(io.BytesIO(img)).convert("RGB")
    if isinstance(img, str):
        if img.startswith("http://") or img.startswith("https://"):
            r = requests.get(img, timeout=30)
            r.raise_for_status()
            return Image.open(io.BytesIO(r.content)).convert("RGB")
        return Image.open(img).convert("RGB")
    if isinstance(img, np.ndarray):
        if img.ndim == 2:  # grayscale
            return Image.fromarray(img).convert("RGB")
        return Image.fromarray(img[..., :3])
    return img  # надеемся, что это уже PIL

def display_images(images, titles=None, figsize=(16, 4)):
    "Отрисовка ряда изображений."
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1: axes = [axes]
    for i, im in enumerate(images):
        axes[i].imshow(to_pil(im))
        axes[i].axis("off")
        if titles and i < len(titles):
            axes[i].set_title(titles[i], fontsize=10)
    plt.tight_layout(); plt.show()

Каждое изображение и текст CLIP превращает в вектор фиксированной длины (512 в случае нашей модели).  
Эти векторы лежат в общем пространстве, где можно сравнивать текст и картинки напрямую.

Чтобы не пересчитывать эмбеддинги заново каждый раз:  
- Считаем их батчами (для ускорения на GPU).  
- Кладём в кэш (например, `.npy` или `.pt` файлы).  

Многие промышленные системы (поиск, рекомендация, deduplication) сначала считают эмбеддинги, а потом работают только с ними в различных дополнительных индексах и сценариях.


In [ ]:
CACHE_DIR = pathlib.Path("./_cache"); CACHE_DIR.mkdir(exist_ok=True)
EMB_CACHE = CACHE_DIR / f"clip_img_emb_vitb32_N{N_SAMPLES}.npy"

def embed_images_clip(images, model, processor, device, batch_size=32):
    model.eval()
    model.to(device)
    feats = []
    with torch.no_grad():
        for i in tqdm(range(0, len(images), batch_size), desc="CLIP image features"):
            batch = [to_pil(im) for im in images[i:i+batch_size]]
            inputs = processor(images=batch, return_tensors="pt", padding=True).to(device)
            out = model.get_image_features(**inputs)
            out = F.normalize(out, p=2, dim=1)  # косинусная геометрия
            feats.append(out.cpu())
    return torch.cat(feats, dim=0).numpy().astype("float32")

# Собираем первые N_SAMPLES изображений (PIL)
images_list = [dataset[i]["image"] for i in range(N_SAMPLES)]

# Загружаем из кэша или считаем
if EMB_CACHE.exists():
    img_feats = np.load(EMB_CACHE)
    if img_feats.shape[0] != N_SAMPLES:
        img_feats = embed_images_clip(images_list, clip_model, clip_processor, device, BATCH_EMB)
        np.save(EMB_CACHE, img_feats)
else:
    img_feats = embed_images_clip(images_list, clip_model, clip_processor, device, BATCH_EMB)
    np.save(EMB_CACHE, img_feats)

assert img_feats.shape[0] == N_SAMPLES
print("Эмбеддинги готовы:", img_feats.shape)


1) Автоматически соберём **топ-категории** из датасета (поля `main_category|category|categories`), c нормализацией текста.  
2) Функция `clip_zeroshot_probs(image, labels)` вернёт распределение по лейблам.  
3) Сконструируем **heatmap**: несколько картинок × несколько категорий.


In [ ]:
def normalize_label(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[_/|]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def collect_top_categories(ds, limit=2000, topk=10):
    counts = collections.Counter()
    n = min(len(ds), limit)
    for i in range(n):
        row = ds[i]
        cat = None
        for key in ("main_category", "category", "categories"):
            if key in row and row[key]:
                cat = row[key]; break
        if isinstance(cat, list) and cat:
            cat = cat[0]
        if isinstance(cat, str):
            counts[normalize_label(cat)] += 1
    top = [c for c,_ in counts.most_common(topk)]
    if not top:
        top = ["electronics","clothing","books","home and garden","sports and outdoors",
               "beauty and personal care","toys and games","automotive","health and household","food and beverages"]
    return top, counts

In [ ]:
product_labels, cat_counts = collect_top_categories(dataset, limit=3000, topk=8)
print("🎯 Категории для zero-shot:", product_labels)

def clip_zeroshot_probs(image, text_labels, model, processor, device):
    pil = to_pil(image)
    inputs = processor(text=text_labels, images=pil, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        out = model(**inputs)
        probs = out.logits_per_image.softmax(dim=-1).cpu().numpy()[0]
    return dict(zip(text_labels, probs))

In [ ]:
# sanity-check
probs0 = clip_zeroshot_probs(images_list[0], product_labels, clip_model, clip_processor, device)
list(probs0.items())[:3]

In [ ]:
# Выбираем несколько индексов равномерно по коллекции
idxs = np.linspace(0, N_SAMPLES-1, num=min(6, N_SAMPLES), dtype=int)
heat = []
for idx in idxs:
    pr = clip_zeroshot_probs(images_list[idx], product_labels, clip_model, clip_processor, device)
    heat.append([pr[l] for l in product_labels])

df_heat = pd.DataFrame(heat, index=[f"img {i}" for i in idxs], columns=product_labels)

display_images([images_list[i] for i in idxs],
               [f"img {i}" for i in idxs],
               figsize=(2.8*len(idxs), 3.2))

plt.figure(figsize=(1.2*len(product_labels)+4, 0.7*len(idxs)+4))
sns.heatmap(df_heat, annot=True, fmt=".2f", cmap="YlOrRd", cbar_kws={'shrink':0.8})
plt.title("Zero-shot: распределение вероятностей по категориям")
plt.xlabel("Категории"); plt.ylabel("Изображения")
plt.xticks(rotation=35, ha="right"); plt.yticks(rotation=0)
plt.tight_layout(); plt.show()

# Faiss индекс

Когда мы имеем тысячи или миллионы изображений, простой перебор (сравнение каждого эмбеддинга с каждым запросом) становится слишком дорогим:  
- При 1 млн объектов нужно сделать 1 млн × 512 операций для каждого запроса.  
- Для 100 млн объектов это уже неподъёмно.  

💡 Решение: использовать **Approximate Nearest Neighbors (ANN)** — алгоритмы приближённого поиска ближайших соседей.  

**FAISS (Facebook AI Similarity Search)** — это библиотека, разработанная в Meta AI, которая:  
- Позволяет строить индексы для эмбеддингов.  
- Ускоряет поиск в сотни раз.  
- Поддерживает как точные (`IndexFlatL2`), так и приближённые (`IVF`, `PQ`, `HNSW`) индексы.  

**Примеры индексов:**
- `IndexFlatL2`: хранит все эмбеддинги, поиск точный, но медленный (O(N)). Хорош для экспериментов и маленьких датасетов.  
- `IndexIVF`: использует «инвертированный файл»: сначала ищем ближайший кластер, затем ближайших соседей внутри него. Значительно быстрее.  
- `IndexPQ`: хранит сжатые вектора, экономит память (важно для 100+ млн объектов).  
- `HNSW`: графовая структура поиска, баланс скорости и качества.  

**Почему это важно:**  
- Именно так работают современные поисковые и рекомендательные системы.  
- CLIP-эмбеддинги → FAISS / HNSW индексы → быстрый поиск картинок/видео/товаров по тексту или любым другим векторным признакам.  

В нашем примере мы построим простой `IndexFlatL2`, чтобы понять механику, а затем можно заменить его на более сложный (IVF/PQ) для больших датасетов.


In [ ]:
import faiss

# Индекс на косинусных (через inner product)
d = img_feats.shape[1]
index = faiss.IndexFlatIP(d)
index.add(img_feats.astype("float32"))
print(f"✅ FAISS: d={d}, ntotal={index.ntotal}")

def search_by_image_idx(query_idx, k=6):
    q = img_feats[query_idx].reshape(1,-1).astype("float32")
    D, I = index.search(q, k)
    return I[0], D[0]

def encode_text_norm(text: str) -> np.ndarray:
    inp = clip_processor(text=[text], return_tensors="pt").to(device)
    with torch.no_grad():
        t = clip_model.get_text_features(**inp)
        t = F.normalize(t, p=2, dim=1)
    return t.cpu().numpy().astype("float32")[0]

def search_by_text(query: str, k=6):
    q = encode_text_norm(query).reshape(1,-1)
    D, I = index.search(q, k)
    return I[0], D[0]

In [ ]:
# image→image
q_idx = int(N_SAMPLES * 0.37)
nbrs, sims = search_by_image_idx(q_idx, k=6)
print("🔍 image→image:", list(zip(nbrs, sims)))
display_images([images_list[q_idx]] + [images_list[i] for i in nbrs],
               ["QUERY"] + [f"sim={s:.2f}" for s in sims], figsize=(18,3))

# text→image
q_text = "traditional percussion instrument"
nbrs_t, sims_t = search_by_text(q_text, k=6)
print("🔎 text→image:", q_text, "→", list(zip(nbrs_t, sims_t)))
display_images([images_list[i] for i in nbrs_t],
               [f"{i}  sim={s:.2f}" for i,s in zip(nbrs_t, sims_t)], figsize=(18,3))

Эмбеддинги в CLIP позволяют производить с ними векторную арифметику, поэтому можно складывать и вычитать смыслы.  
Естественно, это не всегда работает идеально, но даёт интуицию: **пространство эмбеддингов хранит семантику**. Благодаря этому мы можем находить направления, отвечающие за те или иные свойства, что может помочь в дальнейших экспериментах (за рамками данного ноутбука) для контролируемой генерации.


In [ ]:
def vector_arithmetic_search(base: str, minus: str, plus: str, k: int = 6):
    b = encode_text_norm(base)
    m = encode_text_norm(minus)
    p = encode_text_norm(plus)
    q = b - m + p
    q = q / (np.linalg.norm(q) + 1e-9)
    D, I = index.search(q.reshape(1,-1).astype("float32"), k)
    display_images([images_list[i] for i in I[0]],
                   [f"{i}  sim={s:.2f}" for i,s in zip(I[0], D[0])],
                   figsize=(18,3))
    print(f"🧮 '{base}' - '{minus}' + '{plus}'")

In [ ]:
vector_arithmetic_search("electric guitar", "electric", "acoustic", k=6)
vector_arithmetic_search("blue backpack", "blue", "red", k=6)


# Визуализация пространства эмбеддингов

- У CLIP эмбеддинги размерности 512. Визуализировать напрямую невозможно.  
- Используем **UMAP** — метод снижения размерности, который сохраняет локальные структуры (похож на t-SNE, но быстрее и воспроизводим, так как имеет метод `transform`).  
- Получаем 2D-карту, где похожие объекты лежат рядом.  

Когда мы делаем кластеризацию (например, KMeans), возникает вопрос: **сколько кластеров выбрать?**

Для этого используют метрику **Silhouette score**:

- Для каждой точки измеряется:
  - *a* — насколько точка близка к точкам своего кластера.
  - *b* — насколько она далека от точек ближайшего чужого кластера.
- Score = (b - a) / max(a, b).

Интерпретация:
- Ближе к **1** → хороший кластер (точки плотные и далеко от других кластеров).
- Около **0** → точка на границе кластеров.
- Отрицательное значение → точка «ошиблась кластером».

Средний silhouette score помогает подобрать оптимальное число кластеров **K**: там, где метрика максимальна.


In [ ]:
# UMAP
reducer = umap.UMAP(n_components=2, n_neighbors=20, min_dist=0.5, metric="cosine", random_state=SEED)
emb2d = reducer.fit_transform(img_feats)

# Автоподбор K
K_CAND = [5, 8, 10, 12]
best_k, best_score = None, -1
for k in K_CAND:
    km = KMeans(n_clusters=k, n_init=10, random_state=SEED)
    labels = km.fit_predict(emb2d)
    score = silhouette_score(emb2d, labels)
    if score > best_score:
        best_k, best_score = k, score
print(f"Лучший K={best_k} (silhouette={best_score:.3f})")

In [ ]:
km = KMeans(n_clusters=best_k, n_init=20, random_state=SEED).fit(emb2d)
clusters = km.labels_

df2d = pd.DataFrame({
    "x": emb2d[:,0], "y": emb2d[:,1],
    "cluster": clusters,
    "title": [dataset[i].get("title","") for i in range(N_SAMPLES)],
    "idx": np.arange(N_SAMPLES)
})

In [ ]:
# Статический scatter
plt.figure(figsize=(8,6))
for c in range(best_k):
    m = clusters==c
    plt.scatter(df2d.loc[m,"x"], df2d.loc[m,"y"], s=12, alpha=0.7, label=f"C{c}")
plt.legend(title="cluster", ncol=best_k//5+1)
plt.title("UMAP 2D (CLIP image embeddings)")
plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

# Интерактивный Plotly
fig = px.scatter(df2d, x="x", y="y", color=df2d["cluster"].astype(str),
                 hover_data={"idx":True, "title":True, "cluster":True},
                 title="UMAP 2D — интерактивно (наведите курсор)")
fig.update_layout(width=800, height=600, legend_title_text="cluster")
fig.show()

# Интерактивное демо

Здесь мы объединяем всё:  
- Выбираем картинку.  
- Смотрим top-5 категорий (zero-shot).  
- Находим похожие изображения в FAISS.  
- Смотрим на её позицию на UMAP-карте.  

Такой интерактив помогает:  
- лучше понять, как работает модель,  
- почувствовать ограничения (ошибки модели, шумные данные),  
- увидеть практическую ценность мультимодальных эмбеддингов.  


In [ ]:
def clip_topk(image, labels, k=5):
    pr = clip_zeroshot_probs(image, labels, clip_model, clip_processor, device)
    return sorted(pr.items(), key=lambda x: x[1], reverse=True)[:k]

In [ ]:
def interactive_demo(i: int, k_neighbors: int = 8, labels=None, cols: int = 2):
    labels = labels or product_labels
    assert 0 <= i < N_SAMPLES

    top5 = clip_topk(images_list[i], labels, k=5)
    nbrs, sims = search_by_image_idx(i, k=k_neighbors)

    x, y = df2d.loc[i, "x"], df2d.loc[i, "y"]
    cl = int(df2d.loc[i, "cluster"])

    # размеры фигуры: правая часть растягивается под число рядов
    rows = ceil(len(nbrs) / cols)
    H = max(7, 2 + 2.2 * rows)   # высота
    W = 16                       # ширина
    fig = plt.figure(figsize=(W, H))
    gs = GridSpec(2, 3, figure=fig,
                  width_ratios=[1.2, 1.0, 1.4],
                  height_ratios=[1, 1],
                  wspace=0.35, hspace=0.28)

    # левый столбец — исходник
    ax_img = fig.add_subplot(gs[:, 0])
    ax_img.imshow(to_pil(images_list[i])); ax_img.axis("off")
    ax_img.set_title(f"img {i} | cluster={cl}")

    # верх-центр — топ-5 классов
    ax_bar = fig.add_subplot(gs[0, 1])
    labs, probs = zip(*top5)
    ax_bar.barh(range(len(labs)), probs)
    ax_bar.set_yticks(range(len(labs))); ax_bar.set_yticklabels(labs)
    ax_bar.invert_yaxis(); ax_bar.set_xlim(0, 1)
    ax_bar.set_title("Top-5 zero-shot probs")

    # низ-центр — UMAP позиция
    ax_umap = fig.add_subplot(gs[1, 1])
    ax_umap.scatter(df2d["x"], df2d["y"], s=6, alpha=0.15, color="gray")
    ax_umap.scatter([x], [y], s=90, edgecolor="k")
    ax_umap.set_title("UMAP position"); ax_umap.grid(alpha=0.3)

    # правый столбец — грид из соседей (rows x cols)
    right = gs[:, 2].subgridspec(rows, cols, wspace=0.08, hspace=0.12)
    for idx, (j, s) in enumerate(zip(nbrs, sims)):
        r, c = divmod(idx, cols)
        ax = fig.add_subplot(right[r, c])
        ax.imshow(to_pil(images_list[j])); ax.axis("off")
        ax.set_title(f"{j}  sim={s:.2f}", fontsize=9, pad=2)

    plt.tight_layout()
    plt.show()


In [ ]:
interactive_demo(0, k_neighbors=8, cols=2)

In [ ]:
interactive_demo(1000, k_neighbors=8, cols=2)

In [ ]:
interactive_demo(1500, k_neighbors=8, cols=2)